In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../')

In [3]:
from src.utils import loading, Spark
import pyspark.ml as M
import pyspark.sql.functions as F
import pyspark.sql.types as T

In [4]:
spark = Spark()

Spark UI address http://127.0.0.1:4040


In [1]:
from pyspark import SparkContext, SparkConf

In [4]:
conf_spark = SparkConf().set("spark.driver.host", "127.0.0.1")
sc = SparkContext(conf = conf_spark)

In [7]:
conf?

In [8]:
spark.set

In [6]:
datas = loading(spark, '../downloads')

In [7]:
datas

{'links': DataFrame[movieId: string, imdbId: string, tmdbId: string],
 'tags': DataFrame[userId: string, movieId: string, tag: string, timestamp: string],
 'genome-tags': DataFrame[tagId: string, tag: string],
 'ratings': DataFrame[userId: string, movieId: string, rating: string, timestamp: string],
 'genome-scores': DataFrame[movieId: string, tagId: string, relevance: string],
 'movies': DataFrame[movieId: string, title: string, genres: string]}

In [8]:
ratings = datas['ratings']


In [9]:
for key in datas.keys():
    print(key)
    datas[key].show(10)

links
+-------+-------+------+
|movieId| imdbId|tmdbId|
+-------+-------+------+
|      1|0114709|   862|
|      2|0113497|  8844|
|      3|0113228| 15602|
|      4|0114885| 31357|
|      5|0113041| 11862|
|      6|0113277|   949|
|      7|0114319| 11860|
|      8|0112302| 45325|
|      9|0114576|  9091|
|     10|0113189|   710|
+-------+-------+------+
only showing top 10 rows

tags
+------+-------+--------------+----------+
|userId|movieId|           tag| timestamp|
+------+-------+--------------+----------+
|    14|    110|          epic|1443148538|
|    14|    110|      Medieval|1443148532|
|    14|    260|        sci-fi|1442169410|
|    14|    260|  space action|1442169421|
|    14|    318|  imdb top 250|1442615195|
|    14|    318|       justice|1442615192|
|    14|    480|     Dinosaurs|1443148563|
|    14|    593|psychothriller|1444014286|
|    14|   1682|    philosophy|1442615158|
|    14|   1682|  surveillance|1442615167|
+------+-------+--------------+----------+
only showin

In [10]:
movies = datas['movies']

In [11]:
movies.show()

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
|      6|         Heat (1995)|Action|Crime|Thri...|
|      7|      Sabrina (1995)|      Comedy|Romance|
|      8| Tom and Huck (1995)|  Adventure|Children|
|      9| Sudden Death (1995)|              Action|
|     10|    GoldenEye (1995)|Action|Adventure|...|
|     11|American Presiden...|Comedy|Drama|Romance|
|     12|Dracula: Dead and...|       Comedy|Horror|
|     13|        Balto (1995)|Adventure|Animati...|
|     14|        Nixon (1995)|               Drama|
|     15|Cutthroat Island ...|Action|Adventure|...|
|     16|       Casino (1995)|         Crime|Drama|
|     17|Sen

In [12]:
movies.select('genres')

DataFrame[genres: string]

In [13]:
rating_user_count = ratings.groupby('userId').agg(
            F.count(F.col('movieId')).alias('movie_count')
)

rating_item_count = ratings.groupby('movieId').agg(
            F.count(F.col('userId')).alias('user_count')
)

In [14]:
userid = rating_user_count.sort(F.col('movie_count').desc()).limit(20000)

In [15]:
itemid = rating_item_count.sort(F.col('user_count').desc()).limit(1000)

In [16]:
itemid

DataFrame[movieId: string, user_count: bigint]

In [17]:
sample = ratings.join(userid, ratings.userId == userid.userId, 'inner')\
                .select(ratings.userId, ratings.movieId, ratings.rating)\
                .join(itemid, ratings.movieId == itemid.movieId, 'inner')\
                .select(ratings.userId, ratings.movieId, ratings.rating)

In [18]:
sample

DataFrame[userId: string, movieId: string, rating: string]

In [19]:
sample.select('userId').distinct().count()

20000

In [20]:
sample.select('movieId').distinct().count()

1000

In [21]:
sample.count()

6519024

In [22]:
stringIndexer = M.feature.StringIndexer(inputCol='userId', outputCol='userId_indx')
model = stringIndexer.fit(sample)
sample = model.transform(sample)
stringIndexer = M.feature.StringIndexer(inputCol='movieId', outputCol='movieId_indx')
model = stringIndexer.fit(sample)
sample = model.transform(sample)

In [23]:
sample = sample.select('userId', 'movieId', \
                      F.col('userId_indx').cast(T.IntegerType()),
                      F.col('movieId_indx').cast(T.IntegerType()),
                    'rating')\
               .orderBy(['userId_indx', 'movieId_indx'])

In [30]:
sample.coalesce(1).write.csv(path=fp, compression='gzip', mode='overwrite', header = True)

ERROR! Session/line number was not unique in database. History logging moved to new session 29


In [26]:
sample.write.csv?

In [200]:
import pandas as pd

In [201]:
pd.read_csv('sample.csv', engine='python')

IsADirectoryError: [Errno 21] Is a directory: 'sample.csv'